<div style="background-color:#1e1e1e; padding:25px; border-radius:10px; display:table; width:100%;">

  <!-- Left column: text -->
  <div style="display:table-cell; vertical-align:top; width:70%; padding-right:10px;">
    <h1 style="color:#4FC3F7; margin-bottom:10px;">Hands-On Part B: Personalization Methods</h1>

  <p style="font-size:18px; font-style:italic; color:#cccccc;">
    Workshop on T1D Simulator and Digital Twins for Personalized Care
  </p>

  <p style="font-size:15px; line-height:1.6; color:#dddddd;">
fill
  </p>
  </div>

  <!-- Right column: logo -->
  <div style="display:table-cell; vertical-align:middle; text-align:left; width:30%; padding-right:50px;">
    <img src="https://micelab.udg.edu/wp-content/uploads/2022/08/MICElab-letras_png-300x119.png" alt="MiceLab Logo" style="height:80px; border-radius:8px;">
  </div>

</div>


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
0. Imports &amp; Set-Up
</h2>

In [1]:
from pathlib import Path
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tqdm import tqdm
from data_classes import TrainingData
from gan import GANModel

Check GPU availability for TensorFlow, not essential.

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("GPUs:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.13.0
Num GPUs Available: 0
GPUs: []


2025-09-19 11:27:09.588998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [3]:
import os, sys, contextlib, io

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # still useful for later logs
os.environ['ABSL_LOG_LEVEL']      = '3'

@contextlib.contextmanager
def mute_tensorflow_startup():
    """Silence C++ factory/placer warnings emitted during the first TF import."""
    stderr_fileno = sys.stderr.fileno()
    with open(os.devnull, 'w') as devnull:
        saved_stderr = os.dup(stderr_fileno)
        os.dup2(devnull.fileno(), stderr_fileno)   # redirect to /dev/null
        try:
            yield
        finally:
            os.dup2(saved_stderr, stderr_fileno)   # restore stderr
            os.close(saved_stderr)

with mute_tensorflow_startup():
    import tensorflow as tf

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior() # essential for indexing tensors, code crashes without this

<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
1. Load Model & Data
</h2>

Setup path and model name.

In [4]:
path_to_model = "/mnt/c/Users/oriol/Downloads/"
model_name = "pretrained_model.h5"
model_name = "gan_rampedup005999.h5"
model_name = "decoder_epoch_10.h5"

Load the .h5 using the `load_model` method from keras.

In [5]:
model_filename = Path(path_to_model, model_name)
pretrained_vae_model = tf.keras.models.load_model(model_filename)

In [6]:
path_df = Path("data_processed.csv")

df = pd.read_csv(path_df)

display(df.head())

,ID,BG,PI,RA
0,11_0,134.0,0.000000,0.0
1,11_0,125.0,0.000271,0.0
2,11_0,132.0,0.001510,0.0
3,11_0,132.0,0.003488,0.0
4,11_0,132.0,0.006741,0.0


<h2 style="color:#4FC3F7; border-bottom:2px solid #4FC3F7; padding-bottom:4px;">
2. Split Patient Data for Train / Validation
</h2>

In [7]:
split_size = 0.2
df_train, df_val = train_test_split(df, test_size=split_size, random_state=0, shuffle=True)

display(df_train.head())
print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)

,ID,BG,PI,RA
1649,290_0,166.0,1.851760,0.231335
1465,290_0,63.0,0.834074,0.018890
361,11_0,109.0,0.436116,0.000001
315,11_0,93.0,0.275721,0.000246
1372,290_0,131.0,0.855476,0.032081


Train shape: (1382, 4)
Validation shape: (346, 4)


In [8]:
def manual_minmax_scaling(data: np.ndarray, scaler_range: tuple) -> tuple[np.ndarray, float, float]:
    """
    Perform manual min-max scaling on the data.

    Args:
        data (np.ndarray): Data to scale.
    Returns:
        Tuple[np.ndarray, float, float]: Scaled data, minimum value, and maximum value.
    """
    lower_bound = scaler_range[0]
    upper_bound = scaler_range[1]
    min_val = np.min(data)
    max_val = np.max(data)
    scaled_data = lower_bound + (upper_bound - lower_bound) * (data - min_val) / (max_val - min_val)

    return scaled_data, min_val, max_val

def individual_min_max_scaling(
    df: pd.DataFrame, base_output_dir: Path, scaler_range: tuple, apply_bg_log_transformation: bool, gan_inputs: list
) -> pd.DataFrame:
    """
    Perform individual min-max scaling on the dataframe columns.
    Scale all three columns the same way and save only the scalers for 'BG'.

    Args:
        df (pd.DataFrame): Dataframe to scale.
        base_output_dir (Path): Directory to save the scalers.
    Returns:
        pd.DataFrame: Scaled dataframe.
    """
    scalers: dict[str, list[float]] = {'BG_min': [], 'BG_max': []}
    scaled_dfs: list[pd.DataFrame] = []

    for _, group in df.groupby('ID'):
        for column in gan_inputs:
            if column not in ['BG', 'IG']:
                group[f'sc{column}'], _, _ = manual_minmax_scaling(group[column], scaler_range)

        group['scBG'], min_val, max_val = (
            manual_minmax_scaling(group['BG_transformed'], scaler_range)
            if apply_bg_log_transformation
            else manual_minmax_scaling(group['BG'], scaler_range)
        )

        scalers['BG_min'].append(min_val)
        scalers['BG_max'].append(max_val)

        scaled_dfs.append(group)

    scaled_df = pd.concat(scaled_dfs, ignore_index=True)
    scaled_df = scaled_df.sort_values(by=['ID', 'iteration']).reset_index(drop=True)

    save_scalers(pd.DataFrame(scalers), base_output_dir)

    return scaled_df

def save_scalers(scalers_df: pd.DataFrame, base_output_dir: Path) -> None:
    """
    Save the scalers to a file.

    Args:
        scalers (Dict[str, List[float]]): Dictionary containing the scalers.
        base_output_dir (Path): Directory to save the scalers.
    """
    scalers_file: Path = Path(base_output_dir, 'bg_scalers.joblib')
    joblib.dump(scalers_df, scalers_file)

In [9]:
df_scaled = df.copy(deep=True)

df_scaled['iteration'] = df_scaled.groupby('ID').cumcount()
df_scaled = individual_min_max_scaling(df_scaled, 'misc/', (0,1), False, ['BG', 'RA', 'PI'])
df_scaled.drop(columns=['PI', 'RA', 'iteration'], inplace=True)
df_scaled

,ID,BG,scRA,scPI,scBG
0,11_0,134.0,0.000000,0.000000,0.582781
1,11_0,125.0,0.000000,0.000097,0.523179
2,11_0,132.0,0.000000,0.000543,0.569536
3,11_0,132.0,0.000000,0.001253,0.569536
4,11_0,132.0,0.000000,0.002421,0.569536
...,...,...,...,...,...
1723,290_0,70.0,0.035469,0.755634,0.123016
1724,290_0,69.0,0.032020,0.739677,0.119048
1725,290_0,69.0,0.028892,0.724266,0.119048
1726,290_0,69.0,0.026057,0.709428,0.119048


In [10]:
def pack_data(df) -> None:
    """
    Pack the data into vectors for each feature (BG).

    This method creates packed vectors for blood glucose (BG)
    by sliding a window over the time series data. The packed data is then
    stored in new columns in the DataFrame.
    """
    df = df.fillna(0)  # Fill NaN values with 0

    # Pack blood glucose data
    packed_bg_vectors: np.ndarray = np.array(
        [
            df.iloc[i : i + 18]['scBG'].values
            for i in tqdm(range(len(df) - (18 - 1)), desc='Packing BG')
        ]
    )

    df = df.iloc[: len(packed_bg_vectors)]
    df['scBG_packed'] = list(packed_bg_vectors)
    df.reset_index(drop=True, inplace=True)
    return df

In [11]:
df_packed = pack_data(df_scaled)
display(df_packed)

Packing BG: 100%|██████████| 1711/1711 [00:00<00:00, 15957.43it/s]


,ID,BG,scRA,scPI,scBG,scBG_packed
0,11_0,134.0,0.000000,0.000000,0.582781,"[0.5827814569536424, 0.5231788079470199, 0.569..."
1,11_0,125.0,0.000000,0.000097,0.523179,"[0.5231788079470199, 0.5695364238410596, 0.569..."
2,11_0,132.0,0.000000,0.000543,0.569536,"[0.5695364238410596, 0.5695364238410596, 0.569..."
3,11_0,132.0,0.000000,0.001253,0.569536,"[0.5695364238410596, 0.5695364238410596, 0.562..."
4,11_0,132.0,0.000000,0.002421,0.569536,"[0.5695364238410596, 0.5629139072847682, 0.582..."
...,...,...,...,...,...,...
1706,290_0,189.0,0.180982,1.000000,0.595238,"[0.5952380952380952, 0.5515873015873016, 0.515..."
1707,290_0,178.0,0.165737,0.998435,0.551587,"[0.5515873015873016, 0.5158730158730159, 0.5, ..."
1708,290_0,169.0,0.151576,0.993078,0.515873,"[0.5158730158730159, 0.5, 0.47619047619047616,..."
1709,290_0,165.0,0.138455,0.984596,0.500000,"[0.5, 0.47619047619047616, 0.44841269841269843..."


In [12]:
gan_inputs = ['scBG_packed', 'scPI', 'scRA']
data_dict = {column: np.array(df_packed[column].tolist(), dtype=np.float32).reshape(df_packed.shape[0], 1, -1) for column in gan_inputs}

In [13]:
def load_training_data(df):
    train_data_dict = {}
    for input_name in gan_inputs:
        if input_name == 'scBG_packed':
            train_data_dict[f'scBG'] = df['scBG_packed']
        else:
            train_data_dict[f'{input_name}'] = df[f'{input_name}']

    train_data = TrainingData(**train_data_dict)
    return train_data

In [14]:
training_data = load_training_data(data_dict)

In [15]:
gan = GANModel(pretrained_vae_model)

try:
    gan.train(training_data)
except KeyboardInterrupt:
    print('Process interrupted by user')

GAN Training:   4%|▍         | 103/2650 [00:05<01:31, 27.94step/s]

d_loss_real=7.620
d_loss_fake=-52.439
g_loss=39.880, adversarial_loss=39.804, l2_loss=0.075


GAN Training:   8%|▊         | 205/2650 [00:09<01:27, 27.99step/s]

d_loss_real=-21.779
d_loss_fake=-54.201
g_loss=43.021, adversarial_loss=42.940, l2_loss=0.081


GAN Training:  11%|█▏        | 304/2650 [00:12<01:24, 27.74step/s]

...

GAN Training:  94%|█████████▍| 2504/2650 [01:35<00:04, 29.77step/s]

d_loss_real=-89.226
d_loss_fake=-320.500
g_loss=25.606, adversarial_loss=25.368, l2_loss=0.238


GAN Training:  98%|█████████▊| 2604/2650 [01:38<00:01, 29.55step/s]

d_loss_real=-108.232
d_loss_fake=-282.278
g_loss=10.610, adversarial_loss=10.386, l2_loss=0.224


GAN Training: 100%|██████████| 2650/2650 [01:40<00:00, 26.50step/s]
